In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import math
import numpy as np

### Data Setup

In [ ]:
random.seed(42) 
n = 100       # number of seekers
m = 10       # number of banks
d = 3       # dimension of the feature vector
beta = 0.5  # scaling parameter for the exponential transformation

# Create dummy multi-dimensional features for seekers.
seekers = {i: {'features': np.random.uniform(0, 10, d)} for i in range(n)}

# Define each bank's classifier parameters.
banks = {}
for j in range(m):
    w_j = np.random.uniform(0.5, 2.0, d)  # weight vector with random values
    # To ensure rejection, make the bias negative enough so that all f_j(x_i) < 0
    b_j = -np.abs(np.random.uniform(15, 20))  # Set a large negative bias
    banks[j] = {'w': w_j, 'b': b_j}

# Ensure all seekers are rejected by all banks
for i in range(n):
    for j in range(m):
        # Calculate f_j(x_i) = w_j . x_i + b_j for each bank-seeker pair
        f_j_xi = np.dot(banks[j]['w'], seekers[i]['features']) + banks[j]['b']
        
        # Check if the seeker is not rejected by the bank (i.e., f_j(x_i) >= 0)
        if f_j_xi >= 0:
            print(f"Seeker {i} is approved by bank {j}, f_j(x_{i}) = {f_j_xi}. Replacing feature vector.")
            
            # Replace the seeker's feature vector to ensure rejection
            # Set the new feature vector to ensure f_j(x_i) < 0
            new_features = np.random.uniform(0, 10, d)
            while np.dot(banks[j]['w'], new_features) + banks[j]['b'] >= 0:
                new_features = np.random.uniform(0, 10, d)  # keep generating until rejected
            
            seekers[i]['features'] = new_features  # Update feature vector to the new one
            print(f"New features for seeker {i}: {new_features}")
        
        # Print the current f_j(x_i) value to verify rejection
        print(f"f_j(x_{i}) for bank {j}: {f_j_xi}")

capacity = {0: 1, 1: 2}  # Example bank capacities

f_j(x_0) for bank 0: -9.893879712277338
f_j(x_0) for bank 1: -10.09642948795008
f_j(x_1) for bank 0: -12.801881732034596
f_j(x_1) for bank 1: -10.057284397685933
f_j(x_2) for bank 0: -6.419528614167854
f_j(x_2) for bank 1: -5.995040257337555
Seeker 3 is approved by bank 0, f_j(x_3) = 3.582867831987862. Replacing feature vector.
New features for seeker 3: [0.15945136 5.41781486 1.17351884]
f_j(x_3) for bank 0: 3.582867831987862
f_j(x_3) for bank 1: -9.16303015962795
f_j(x_4) for bank 0: -15.201330359471008
f_j(x_4) for bank 1: -16.23941285510294


### Recourse Cost and Weight Functions

In [95]:
def recourse_cost(seeker, bank):
    """
    Computes the minimum change required for a seeker to be approved by a bank.
    For a feature vector x and a linear classifier f(x)=w.x+b,
    the minimal recourse cost (L2 norm) is given by:
        cost = max(0, (- (w.x + b)) / ||w||)
    """
    x = seeker['features']
    w = bank['w']
    b = bank['b']
    f_x = np.dot(w, x) + b
    norm_w = np.linalg.norm(w)
    cost = (-f_x) / norm_w
    return max(0, cost)

def weight_from_cost(cost, beta):
    """
    Transforms the cost into a weight using the exponential transformation.
    """
    return math.exp(-beta * cost)

# Compute recourse cost and corresponding weight for each (seeker, bank) pair.
costs = {}
weights = {}
for i in range(n):
    for j in range(m):
        cost_ij = recourse_cost(seekers[i], banks[j])
        costs[i, j] = cost_ij
        weights[i, j] = weight_from_cost(cost_ij, beta)

### Gurobi Model Setup

In [97]:
model = gp.Model("LoanAssignment_Recourse_MultiDim")

# Create binary decision variables x[i,j] indicating if seeker i is assigned to bank j.
z = {}
for i in range(n):
    for j in range(m):
        z[i, j] = model.addVar(vtype=GRB.BINARY, name=f"z_{i}_{j}")

# Set the objective: maximize total weight of the assignments.
model.setObjective(gp.quicksum(weights[i, j] * z[i, j] for i in range(n) for j in range(m)), GRB.MAXIMIZE)

# Constraints:
# 1. Each seeker is assigned to at most one bank.
for i in range(n):
    model.addConstr(gp.quicksum(z[i, j] for j in range(m)) <= 1, name=f"seeker_{i}")

# 2. Each bank's assignments do not exceed its capacity.
for j in range(m):
    model.addConstr(gp.quicksum(z[i, j] for i in range(n)) <= capacity[j], name=f"bank_{j}")

# Optimize the model.
model.optimize()



Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x4796b7bf
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 0.4633141
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 4.860379e-01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.4860379    0.486

### Output the Solution and Recourse Recommendations

In [98]:
if model.status == GRB.OPTIMAL:
    print("\nOptimal assignment and recommended recourse actions:")
    for i in range(n):
        for j in range(m):
            if z[i, j].X > 0.5:
                action = costs[i, j]  # minimal change required
                print(f"Seeker {i} (features={seekers[i]['features']}) assigned to Bank {j} "
                      f"(classifier: w={banks[j]['w']}, b={banks[j]['b']:.2f})")
                print(f"  Recourse cost (minimal change required): {action:.2f}")
                # print(f"  Transformed weight: {weights[i, j]:.4f}\n")
else:
    print("No optimal solution found.")


Optimal assignment and recommended recourse actions:
Seeker 0 (features=[1.35867547 3.93695308 1.43943149]) assigned to Bank 0 (classifier: w=[1.45936863 1.59361342 0.58684098], b=-19.00)
  Recourse cost (minimal change required): 4.42
Seeker 2 (features=[1.98946701 4.68180837 3.76848206]) assigned to Bank 1 (classifier: w=[1.06205029 1.65962486 0.94256285], b=-19.43)
  Recourse cost (minimal change required): 2.74
Seeker 3 (features=[0.15945136 5.41781486 1.17351884]) assigned to Bank 1 (classifier: w=[1.06205029 1.65962486 0.94256285], b=-19.43)
  Recourse cost (minimal change required): 4.20


### Global Optimum

In [101]:
# Compute the global optimum (sum of maximum weights for all seekers without any capacity constraints)
global_optimum = 0

# For each seeker, choose the bank with the highest weight (best assignment) and sum them
for i in range(n):
    max_weight = max(weights[i, j] for j in range(m))  # Find the maximum weight for seeker i
    global_optimum += max_weight  # Add it to the global optimum sum

print(f"Global Optimum (without capacity constraints): {global_optimum}")

Global Optimum (without capacity constraints): 0.6196269818527836
